[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/databyjp/AcademyXI_DA/blob/main/notebooks/AcademyXi_DA_Module_7_data_analysis_1.ipynb)

## AcademyXi Data Analysis - Data Analysis 1: 
### Workshop - Data analysis with Python
In this workshop module, we will show you how to use Python to perform some the data analysis tasks you've already seen.

In [1]:
# Install additional libraries required (fsspec and s3fs) to load files through AWS S3
%%capture tmp
!pip install fsspec s3fs

# Import libraries to be used
import pandas as pd
import numpy as np
import plotly.express as px

#### Visualising distributions

Below reproduces many of the charts from earlier in the module with Plotly. Although you have already seen how to produce statistical visualisations such as histograms, these may be useful as revision material.

In [2]:
df = pd.read_csv("s3://databyjp/academyxi/wk7_physPerformance.csv")

In [3]:
df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [4]:
# Histogram - see https://plotly.com/python/histograms/
fig = px.histogram(df, x="height_cm", nbins=40)
fig.show()

In [5]:
fig = px.histogram(df, x="gripForce", nbins=40)
fig.show()

In [6]:
fig = px.histogram(df, x="gripForce", nbins=40, color="gender")
fig.show()

In [7]:
# Overlaid histogram - https://plotly.com/python/histograms/#overlaid-histogram
fig = px.histogram(df, x="gripForce", nbins=40, color="gender")
fig.update_layout(barmode='overlay')  
fig.update_traces(opacity=0.75)
fig.show()

In [8]:
fig = px.scatter(df, x="weight_kg", y="height_cm")
fig.show()

In [9]:
fig = px.scatter(df, x="weight_kg", y="broad jump_cm")
fig.show()

In [10]:
fig = px.scatter(df, x="weight_kg", y="broad jump_cm", color="gender")
fig.show()

#### Scatter plot matrices (aka Pairplots)

Now that you know how to use scatter plots to identify correlations, you may wish to start visualising a number of combinations of variables. 

This task is made easier with tools which can produce scatter plot matrics, also known as pairplots.

- https://plotly.com/python/splom/
- https://seaborn.pydata.org/generated/seaborn.pairplot.html

See an example below

In [11]:
fig = px.scatter_matrix(df,
                        dimensions=["age", "height_cm", "diastolic", "gripForce", "broad jump_cm"], 
                        color="gender")
fig.show()

Although crude due to resolution limitations, these plots give some idea of correlation between variables at a glance, which may help to narrow down your search.

In almost all visualisation tools, you can adjust marker sizes. Where there are too many overlapping markers, plotting a randomly sampled subset may also help to reveal global patterns.

In [12]:
sm_df = df.sample(frac=0.05)  # Create a dataframe with only a small portion of the full size dataframe
fig = px.scatter_matrix(sm_df,
                        dimensions=["age", "height_cm", "diastolic", "gripForce", "broad jump_cm"], 
                        color="gender")
fig.update_traces(marker=dict(size=2))  # Set a small size for the markers to better see patterns
fig.show()

## Grouping data

In pandas, grouping data is a two-step process comprising of using the `groupby` method to create a grouped data object, and then using one or more aggregation functions such as:
- `count`
- `sum`
- `min`
- `max`

To generate summaries of the grouped data. The resulting outputs are similar to what is possible through pivot tables.

Let's now open the Melbourne housing dataset which you would have seen before:

In [13]:
df = pd.read_csv("s3://databyjp/academyxi/wk7_melbourne_housing.csv")

In [14]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


We see in our casual inspection that the `Price` column includes null values. Let's clean the data by dropping those rows.

In [15]:
df = df[df["Price"].notna()]

Now, we can find which suburb's listings have the most or fewest rooms on average. This can be done by:
- Grouping the data by `Suburb` column, and
- Taking the mean of the values

In [16]:
grp_df = df.groupby("Suburb")  # Group the dataframe
mean_rms = grp_df.mean()["Rooms"]  # Take a mean
print(mean_rms.sort_values().round(2))  # Sort by values, round to 2 decimal places, then print to screen

Suburb
East Melbourne    1.97
Melbourne         1.98
Ripponlea         2.00
St Kilda          2.02
South Yarra       2.05
                  ... 
Derrimut          4.07
Attwood           4.15
Warranwood        4.33
Wattle Glen       4.50
Wildwood          5.00
Name: Rooms, Length: 345, dtype: float64


Now see if you can do the same, but get the mean prices, grouped by number of rooms

In [17]:
# How would you modify the below to group the data by rooms, and print mean prices?
# Note: .astype(int) converts the resulting data to integers
print(df.groupby("Suburb").mean()["Rooms"].sort_values().astype(int))

Suburb
East Melbourne    1
Melbourne         1
Ripponlea         2
St Kilda          2
South Yarra       2
                 ..
Derrimut          4
Attwood           4
Warranwood        4
Wattle Glen       4
Wildwood          5
Name: Rooms, Length: 345, dtype: int64


To group the data by multiple columns, the input should be a python list as shown below. This will produce a grouped pandas series with a nested index (multi-index).

In [18]:
tmp_ser = df.groupby(["Bedroom2", "Bathroom"]).mean()["Price"].astype(int)

In [19]:
tmp_ser.head()

Bedroom2  Bathroom
0.0       0.0          972333
          1.0          805583
          2.0         1596666
          3.0         1026000
1.0       1.0          437134
Name: Price, dtype: int64

The series can be converted a dataframe by the `.reset_index` method:

In [20]:
grp_df = tmp_ser.reset_index()
grp_df.head()

,Bedroom2,Bathroom,Price
0,0.0,0.0,972333
1,0.0,1.0,805583
2,0.0,2.0,1596666
3,0.0,3.0,1026000
4,1.0,1.0,437134


It can be filtered just as we had previously done.

In [21]:
grp_df = grp_df[(grp_df["Bathroom"] > 0) & (grp_df["Bedroom2"] <= 3)]

In [22]:
grp_df

,Bedroom2,Bathroom,Price
1,0.0,1.0,805583
2,0.0,2.0,1596666
3,0.0,3.0,1026000
4,1.0,1.0,437134
5,1.0,2.0,1147250
6,1.0,3.0,1650000
8,2.0,1.0,783268
9,2.0,2.0,931931
10,2.0,3.0,1285961
12,3.0,1.0,982931


#### Data bins

Bins of data can be created by pandas' `cut` or `qcut` methods to convert continuous variables to discrete ones. `cut` will create bins of given sizes, whereas `qcut` will create  bins based on number of observations in each bin.

Documentation:
- https://pandas.pydata.org/docs/reference/api/pandas.cut.html
- https://pandas.pydata.org/docs/reference/api/pandas.qcut.html

In [23]:
cut_bins = [i for i in range(0, 51, 5)]  # Create bins (0 to 50 in steps of 5) 
dist_bins = pd.cut(df["Distance"], bins=cut_bins)
df = df.assign(dist_bins=dist_bins)

In [24]:
df.groupby("dist_bins").mean()["Price"].astype(int)

dist_bins
(0, 5]      1147319
(5, 10]     1179721
(10, 15]    1017543
(15, 20]     922968
(20, 25]     776678
(25, 30]     660634
(30, 35]     556808
(35, 40]     731303
(40, 45]     573409
(45, 50]     693776
Name: Price, dtype: int64

## Multiple aggregations

For aggregate multiple columns, pandas provides the `agg` method which allows aggregating different columns as well as different methods within. 

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html

In [25]:
df.groupby("dist_bins").agg({"Price": "mean", "Suburb": "count"}).astype(int)

,Price,Suburb
dist_bins,,
"(0, 5]",1147319,3834
"(5, 10]",1179721,8950
"(10, 15]",1017543,8989
"(15, 20]",922968,2830
"(20, 25]",776678,1368
"(25, 30]",660634,553
"(30, 35]",556808,335
"(35, 40]",731303,240
"(40, 45]",573409,33


## Combining tables

Pandas provides functionalities to allow multiple tables to be joined. The main method with which to do this is its `merge` method. 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

Take a look at the below examples:

In [ ]:
pop_df = pd.read_csv("s3://databyjp/academyxi/wk7_wdi_pop_simple.csv")
co2_df = pd.read_csv("s3://databyjp/academyxi/wk7_wdi_co2_simple.csv")

In [ ]:
pop_df.head()

In [ ]:
co2_df.head()

To create an inner join on the `Country Code` column, the following syntax can be used.

In [ ]:
merged_df = pop_df.merge(co2_df, how="inner", on="Country Code")

In [ ]:
merged_df.head()

Now, let's see what would happen with dataframes containing slightly different indices. This can be created in our case by dropping null data rows, as each dataset is missing data for different countries.

Once the missing data has been dropped, we can see that the number of observations are different.

In [ ]:
co2_df = co2_df[co2_df["CO2 emissions (kt)"].notna()]
pop_df = pop_df[pop_df["Population, total"].notna()]
print(f"co2_df is {len(co2_df)} rows long")
print(f"pop_df is {len(pop_df)} rows long")

Now, take a look at the differences produced by:
- Inner join,
- Left join,
- Right join, and
- Outer join

In [ ]:
inner_merged_df = pop_df.merge(co2_df, how="inner", on="Country Code")

In [ ]:
inner_merged_df.head()

In [ ]:
left_merged_df = pop_df.merge(co2_df, how="left", on="Country Code")

In [ ]:
left_merged_df.head()

In [ ]:
right_merged_df = pop_df.merge(co2_df, how="right", on="Country Code")

In [ ]:
right_merged_df.head()

In [ ]:
outer_merged_df = pop_df.merge(co2_df, how="outer", on="Country Code")

In [ ]:
outer_merged_df.head()

Now we can check the number of rows in each dataframe:

In [ ]:
print(f"Inner merged dataframe: {len(inner_merged_df)} rows long")
print(f"Left merged dataframe: {len(left_merged_df)} rows long")
print(f"Right merged dataframe: {len(right_merged_df)} rows long")
print(f"Outer merged dataframe: {len(outer_merged_df)} rows long")

You can see that inner join only includes rows which are present in both dataframes, while left/right join only includes rows of the respective dataframes. 

The outer join includes rows that are present in at least one of the two dataframes.